# *Homo Silicus* Replication
This notebook contains code to replicate the data from Large Language Models as Simulated Economic Agents: What Can We Learn from *Homo Silicus*? <a href="https://john-joseph-horton.com/papers/llm_ask.pdf" style="color:#4e4089">(Horton (2023))</a> in `edsl`.


Replications are presented in the same order as the paper.
Note that results may vary given that this notebook uses GPT-3.5 and GPT-4 and the original paper used davinci-003, which is no longer available.

1. Understanding Social Preferences with Simple Tests <a href="https://www.jstor.org/stable/4132490" style="color:#4e4089">Charness and Rabin (2002)</a>
2. Fairness as a Constraint on Profit Seeking: Entitlements in the Market <a href="https://www.jstor.org/stable/1806070" style="color:#4e4089">(Kahneman et al. (1986))</a>
3. Understanding Social Preferences with Simple Tests <a href="https://scholar.harvard.edu/sites/scholar.harvard.edu/files/rzeckhauser/files/status_quo_bias_in_decision_making.pdf" style="color:#4e4089">(Samuelson and Zeckhauser(1988))</a>
4. Price Floors and Employer Preferences: Evidence from a Minimum Wage Experiment <a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2898827" style="color:#4e4089">(Horton (2023))</a>

# A social preferences experiment
Understanding Social Preferences with Simple Tests <a href="https://www.jstor.org/stable/4132490" style="color:#4e4089">(Charness and Rabin (2002))</a>

In [1]:
from edsl.questions import QuestionMultipleChoice
from edsl.scenarios.Scenario import Scenario
from edsl.language_models import LanguageModelOpenAIFour
from edsl.agents import Agent

Setup agents with different preferences

In [2]:
personalities = [
    "You only care about the total pay-off of both players",
    "You only care about fairness between players",
    "You only care about your own pay-off",
    ""
    ]

agents = [Agent(traits = {'Your preferences': p}) for p in personalities]
agents

[Agent(traits = {'Your preferences': 'You only care about the total pay-off of both players'}),
 Agent(traits = {'Your preferences': 'You only care about fairness between players'}),
 Agent(traits = {'Your preferences': 'You only care about your own pay-off'}),
 Agent(traits = {'Your preferences': ''})]

You can randomize over a set of nested python objects to "link" different variables that you want in a specific combination. 

In [3]:
q = QuestionMultipleChoice(
question_name = "charness_rabin",
question_text = """
You are deciding on allocation for yourself and another person, Person A. 

Option Left:  You get ${{choice[0][1]}}, Person A gets ${{choice[0][0]}}
Option Right: You get ${{choice[1][1]}}, Person A gets ${{choice[1][0]}}

What do you choose, with one word [Left, Right]?
""",	
question_options = ["Left", "Right"]	
)
q

QuestionMultipleChoice(question_name = 'charness_rabin', short_names_dict = {}, question_text = '
You are deciding on allocation for yourself and another person, Person A. 

Option Left:  You get ${{choice[0][1]}}, Person A gets ${{choice[0][0]}}
Option Right: You get ${{choice[1][1]}}, Person A gets ${{choice[1][0]}}

What do you choose, with one word [Left, Right]?
', question_options = ['Left', 'Right'])

Now, instead of the factorial combination of all values in the `choices` list, the experiment will only be run over the 6 choice options putting each choice in the correct variable slot in the question!

In [4]:
choices = list({
    ((400, 400), (750, 400)),
    ((400, 400), (750, 375)),
    ((800, 200), (0, 0)),
    ((300, 600), (700, 500)),
    ((200, 700), (600, 600)),
    ((0, 800), (400, 400))
})


scenarios = [Scenario({"choice":c}) for c in choices]
scenarios

[{'choice': ((300, 600), (700, 500))},
 {'choice': ((400, 400), (750, 400))},
 {'choice': ((0, 800), (400, 400))},
 {'choice': ((400, 400), (750, 375))},
 {'choice': ((800, 200), (0, 0))},
 {'choice': ((200, 700), (600, 600))}]

In [5]:
gpt4 = LanguageModelOpenAIFour(temperature = 0)
responses = q.by(scenarios).by(agents).by(gpt4).run(method ='threaded')

# can convert to pandas easily to do analysis!
responses.to_pandas().head(5)

|; 100% of tasks complete; 1 worker(s); Est. TPM (k):  0 (0% of lim.); Est. RPM (k): 0 (0% of lim.)); 

,model.temperature,model.max_tokens,scenario.choice,model.top_p,answer.charness_rabin,model.use_cache,answer.charness_rabin_comment,agent.your_preferences,model.model,model.presence_penalty,model.frequency_penalty
0,0,1000,"((300, 600), (700, 500))",1,Right,True,Choosing Right results in a higher total pay-o...,You only care about the total pay-off of both ...,gpt-4-1106-preview,0,0
1,0,1000,"((400, 400), (750, 400))",1,Right,True,Choosing Right maximizes the total pay-off for...,You only care about the total pay-off of both ...,gpt-4-1106-preview,0,0
2,0,1000,"((0, 800), (400, 400))",1,Right,True,Choosing Right because it maximizes the total ...,You only care about the total pay-off of both ...,gpt-4-1106-preview,0,0
3,0,1000,"((400, 400), (750, 375))",1,Right,True,Choosing Right maximizes the total pay-off for...,You only care about the total pay-off of both ...,gpt-4-1106-preview,0,0
4,0,1000,"((800, 200), (0, 0))",1,Left,True,Choosing Left maximizes the total pay-off for ...,You only care about the total pay-off of both ...,gpt-4-1106-preview,0,0


# Fairness as a constraint on profit-seeking
Fairness as a Constraint on Profit Seeking: Entitlements in the Market <a href="https://www.jstor.org/stable/1806070" style="color:#4e4089">(Kahneman et al. (1986))</a>

In [6]:
from edsl.questions import QuestionMultipleChoice
from edsl.scenarios.Scenario import Scenario
from edsl.agents import Agent
from edsl.language_models import LanguageModelOpenAIFour, LanguageModelOpenAIThreeFiveTurbo

In [7]:

political_views = ["socialist", "leftist","liberal","moderate","liberterian", "conservative"]
agents = [Agent(traits = {'Your political views': f"You are a {p}"}) for p in political_views ]
agents

[Agent(traits = {'Your political views': 'You are a socialist'}),
 Agent(traits = {'Your political views': 'You are a leftist'}),
 Agent(traits = {'Your political views': 'You are a liberal'}),
 Agent(traits = {'Your political views': 'You are a moderate'}),
 Agent(traits = {'Your political views': 'You are a liberterian'}),
 Agent(traits = {'Your political views': 'You are a conservative'})]

In [8]:
# Construct a question
q = QuestionMultipleChoice(
question_name = "shovel",
question_text = """A hardware store has been selling snow shovels for $15. 
The morning after a large snowstorm, the store {{store_action}} ${{new_price}}.
How would you rate this action?
""",	
question_options = ["Completely Fair", "Acceptable", "Unfair", "Very Unfair"]	
)
q


QuestionMultipleChoice(question_name = 'shovel', short_names_dict = {}, question_text = 'A hardware store has been selling snow shovels for $15. 
The morning after a large snowstorm, the store {{store_action}} ${{new_price}}.
How would you rate this action?
', question_options = ['Completely Fair', 'Acceptable', 'Unfair', 'Very Unfair'])

Generates all possible combinations of the scenario (4 x 2 = 8)

In [9]:
new_prices = [16, 20, 40, 100]
store_actions = ["changes the price to", "raises the price to"]
scenarios = [Scenario({"new_price":np,"store_action":sa}) for np in new_prices for sa in store_actions]
scenarios


[{'new_price': 16, 'store_action': 'changes the price to'},
 {'new_price': 16, 'store_action': 'raises the price to'},
 {'new_price': 20, 'store_action': 'changes the price to'},
 {'new_price': 20, 'store_action': 'raises the price to'},
 {'new_price': 40, 'store_action': 'changes the price to'},
 {'new_price': 40, 'store_action': 'raises the price to'},
 {'new_price': 100, 'store_action': 'changes the price to'},
 {'new_price': 100, 'store_action': 'raises the price to'}]

Questions can be asked two multiple LLMs and api calls can be threaded (parallelized) for speed

In [10]:
# optional specify temperature
gpt4 = LanguageModelOpenAIFour(temperature = 0)
gpt35 = LanguageModelOpenAIThreeFiveTurbo(temperature = 0)
responses = q.by(scenarios).by(agents).by(gpt35,gpt4).run(method = 'threaded')

responses.to_pandas().head(5)


|; 100% of tasks complete; 2 worker(s); Est. TPM (k):  0 (0% of lim.); Est. RPM (k): 0 (0% of lim.)); 

,model.temperature,scenario.new_price,model.max_tokens,answer.shovel_comment,model.top_p,answer.shovel,scenario.store_action,model.use_cache,model.model,agent.your_political_views,model.presence_penalty,model.frequency_penalty
0,0,16,1000,This action is unfair. The store is taking adv...,1,Unfair,changes the price to,True,gpt-3.5-turbo,You are a socialist,0,0
1,0,16,1000,While it is understandable that demand for sno...,1,Unfair,changes the price to,True,gpt-4-1106-preview,You are a socialist,0,0
2,0,16,1000,Raising the price of snow shovels after a larg...,1,Very Unfair,raises the price to,True,gpt-3.5-turbo,You are a socialist,0,0
3,0,16,1000,The price increase on snow shovels immediately...,1,Unfair,raises the price to,True,gpt-4-1106-preview,You are a socialist,0,0
4,0,20,1000,This action is very unfair. The hardware store...,1,Very Unfair,changes the price to,True,gpt-3.5-turbo,You are a socialist,0,0


# Status Quo Bias in Decision Making
Understanding Social Preferences with Simple Tests <a href="https://scholar.harvard.edu/sites/scholar.harvard.edu/files/rzeckhauser/files/status_quo_bias_in_decision_making.pdf" style="color:#4e4089">(Samuelson and Zeckhauser(1988))</a>


In [11]:
from edsl.agents.Agent import Agent
from edsl.questions import QuestionMultipleChoice
from edsl.language_models import LanguageModelOpenAIFour
from edsl.surveys import Survey

Creating the first question with a neutral framing tha has a static set of answers

In [12]:
q_neutral = QuestionMultipleChoice(
question_name = "zeckhauser_neutral",
question_text = """
The National Highway Safety Commission is deciding how to allocate its budget between two safety research programs: 
i) improving automobile safety (bumpers, body, gas tank configurations, seatbelts) and 
ii) improving the safety of interstate highways (guard rails, grading, highway interchanges, and implementing selective reduced speed limits).
Please choose your most preferred option.
""",	
question_options = ["Allocate 70% to auto safety and 30% to highway safety", "Allocate 30% to auto safety and 70% to highway safety",
                    "Allocate 60% to auto safety and 40% to highway safety", "Allocate 50% to auto safety and 50% to highway safety"]	
)
q_neutral

QuestionMultipleChoice(question_name = 'zeckhauser_neutral', short_names_dict = {}, question_text = '
The National Highway Safety Commission is deciding how to allocate its budget between two safety research programs: 
i) improving automobile safety (bumpers, body, gas tank configurations, seatbelts) and 
ii) improving the safety of interstate highways (guard rails, grading, highway interchanges, and implementing selective reduced speed limits).
Please choose your most preferred option.
', question_options = ['Allocate 70% to auto safety and 30% to highway safety', 'Allocate 30% to auto safety and 70% to highway safety', 'Allocate 60% to auto safety and 40% to highway safety', 'Allocate 50% to auto safety and 50% to highway safety'])

We can create different versions of a question and edit the answers with a helper function. Allows us to flexibly accomodate the different status quo framings

In [13]:

#helper function
def generate_answer(status_quo, auto):
    '''
    status_quo: int, percentage of budget allocated currently allocated to highway program
    auto: int, percentage of budget proposed to be allocated to highway program

    returns: str
    '''
    if status_quo > auto:
        return f"Decrease auto program by {status_quo - auto}% of budget and raise the highway program by like amount"
    if status_quo == auto:
        return f"Maintain present budget amounts for the programs"
    if status_quo < auto:
        return f"Decrease the highway program by {auto - status_quo}% of budget and raise the auto program by like amount"
    


Now we can just iterate through the different status quo framings and generate the appropriate question and answer options. We save the questions in a list to create a survey in the next step.

In [14]:
questions = [q_neutral]
auto_vals = []
for status_quo_auto in [70, 30, 60, 50]:
    auto_vals += [[status_quo_auto, 100-status_quo_auto]]
    question = QuestionMultipleChoice(
                    question_name = f"zeckhauser_{status_quo_auto}",
                    question_text = f"""
The National Highway Safety Commission is deciding how to allocate its budget between two safety research programs: 
i) improving automobile safety (bumpers, body, gas tank configurations, seatbelts) and 
ii) improving the safety of interstate highways (guard rails, grading, highway interchanges, and implementing selective reduced speed limits).
The current budget allocation is {status_quo_auto}% to auto safety and {100-status_quo_auto}% to highway safety.
Please choose your most preferred option.
""",	
                    question_options = [generate_answer(status_quo_auto, 70), generate_answer(status_quo_auto, 30),
                            generate_answer(status_quo_auto, 60), generate_answer(status_quo_auto, 50)]	
)   
    questions += [question]

questions


[QuestionMultipleChoice(question_name = 'zeckhauser_neutral', short_names_dict = {}, question_text = '
 The National Highway Safety Commission is deciding how to allocate its budget between two safety research programs: 
 i) improving automobile safety (bumpers, body, gas tank configurations, seatbelts) and 
 ii) improving the safety of interstate highways (guard rails, grading, highway interchanges, and implementing selective reduced speed limits).
 Please choose your most preferred option.
 ', question_options = ['Allocate 70% to auto safety and 30% to highway safety', 'Allocate 30% to auto safety and 70% to highway safety', 'Allocate 60% to auto safety and 40% to highway safety', 'Allocate 50% to auto safety and 50% to highway safety']),
 QuestionMultipleChoice(question_name = 'zeckhauser_70', short_names_dict = {}, question_text = '
 The National Highway Safety Commission is deciding how to allocate its budget between two safety research programs: 
 i) improving automobile safety (

Generating the agents with different preferences about what they care about

In [15]:
### agents
options_sets = [("car", "highway"), ("highway", "car")]
phrases = [
    "{} safety is the most important thing.",
    "{} safety is a terrible waste of money; we should only fund {} safety.",
    "{} safety is all that mataters. We should not fund {} safety.",
    "{} safety and {} safety are equally important.",
    "{} safety is slightly more important than {} safety.",
    "I don't really care about {} safety or {} safety."
]

#helper function to make first letter capital
def capitalize_first_letter(s):
    return s[0].upper() + s[1:]

views = [capitalize_first_letter(phrase.format(option1, option2)) for option1, option2 in options_sets for phrase in phrases]

agents = [Agent(traits = {'Your views':view}) for view in views]
agents

[Agent(traits = {'Your views': 'Car safety is the most important thing.'}),
 Agent(traits = {'Your views': 'Car safety is a terrible waste of money; we should only fund highway safety.'}),
 Agent(traits = {'Your views': 'Car safety is all that mataters. We should not fund highway safety.'}),
 Agent(traits = {'Your views': 'Car safety and highway safety are equally important.'}),
 Agent(traits = {'Your views': 'Car safety is slightly more important than highway safety.'}),
 Agent(traits = {'Your views': "I don't really care about car safety or highway safety."}),
 Agent(traits = {'Your views': 'Highway safety is the most important thing.'}),
 Agent(traits = {'Your views': 'Highway safety is a terrible waste of money; we should only fund car safety.'}),
 Agent(traits = {'Your views': 'Highway safety is all that mataters. We should not fund car safety.'}),
 Agent(traits = {'Your views': 'Highway safety and car safety are equally important.'}),
 Agent(traits = {'Your views': 'Highway safet

Putting the questions in a survey to run them by the agents. Creating multiple "almost-zero" temperature versions of gpt-4 to increase the sample size without hitting the cache

In [16]:

survey = Survey(
    questions = questions,
)

models = [LanguageModelOpenAIFour(temperature=0 + e/1000000) for e in range(3)]

responses = survey.by(agents).by(models).run(method = 'threaded')

|; 100% of tasks complete; 1 worker(s); Est. TPM (k):  0 (0% of lim.); Est. RPM (k): 0 (0% of lim.)); 


# Labor-labor substitution in the presence of a minimum wage
Price Floors and Employer Preferences: Evidence from a Minimum Wage Experiment <a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2898827" style="color:#4e4089">(Horton (2023))</a>


In [17]:
from edsl.questions import QuestionMultipleChoice
from edsl.scenarios.Scenario import Scenario
from edsl.language_models import LanguageModelOpenAIFour, LanguageModelOpenAIThreeFiveTurbo
m4 = LanguageModelOpenAIFour(temperature = 0)
m35 = LanguageModelOpenAIThreeFiveTurbo(temperature = 0)

In [18]:
# Construct a question
q = QuestionMultipleChoice(
question_name = "horton_wage",
question_text = """
You are hiring for the role of “Dishwasher.”
The typical hourly rate is $12/hour.
You have 2 candidates. 
Person 1: Has 1 year(s) of experience in this role. Requests ${{pay1}}/hour.
Person 2: Has 0 year(s) of experience in this role. Requests ${{pay2}}/hour. 
Who would you hire? You have to pick one.
""",	
question_options = ["Person 1", "Person 2"]	
)
q

QuestionMultipleChoice(question_name = 'horton_wage', short_names_dict = {}, question_text = '
You are hiring for the role of “Dishwasher.”
The typical hourly rate is $12/hour.
You have 2 candidates. 
Person 1: Has 1 year(s) of experience in this role. Requests ${{pay1}}/hour.
Person 2: Has 0 year(s) of experience in this role. Requests ${{pay2}}/hour. 
Who would you hire? You have to pick one.
', question_options = ['Person 1', 'Person 2'])

Running the survey and printing in a nice format

In [19]:
pay1 = [13, 14, 15, 16, 17, 18, 19]
pay2 = [13, 15]

#making the sample size bigger
models = [LanguageModelOpenAIFour(temperature=0 + e/1000000) for e in range(5)] + [LanguageModelOpenAIThreeFiveTurbo(temperature=0 + e/1000000) for e in range(5)]

scenarios = [Scenario({"pay1":p1, "pay2":p2}) for p1 in pay1 for p2 in pay2]
responses = q.by(scenarios).by(models).run(method = 'threaded')
(responses.select("scenario.*", "horton_wage").print(pretty_labels={
        "scenario.pay1":"Person 1 Requests",
        "scenario.pay2":"Person 2 Requests",
     "answer.horton_wage":"Selected Person",
 }))

|; 100% of tasks complete; 2 worker(s); Est. TPM (k):  0 (0% of lim.); Est. RPM (k): 0 (0% of lim.)); 

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Person 2 Requests ┃ Person 1 Requests ┃ Selected Person ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 13                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 13                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 14                │ Person 1        │
├───────────────────┼───────────────────┼─────────────────┤
│ 15                │ 14                │ Person 1        │
├───────────────────┼───────────────────